# Reference

http://geospatialpython.com/2011/02/clip-raster-using-shapefile.html

http://gis.stackexchange.com/questions/145015/is-it-possible-to-look-at-the-contents-of-shapefile-using-python-without-an-arcm/145029

http://gis.stackexchange.com/questions/95160/union-multiple-shapefiles-with-overlapping-shapes

http://stackoverflow.com/questions/14211340/automatically-cropping-an-image-with-python-pil

http://stackoverflow.com/questions/15403580/extracting-a-subimage-in-shape-of-a-polygon

In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import operator
from osgeo import gdal, gdalnumeric, ogr, osr
import Image, ImageDraw
# import geopandas as gp
import json

In [3]:
# raster = '../../../Datasets/Fusion/FS2_G005_MS_L4f_20150122_merge2_ot_NC.tif'
# # raster = '../../../tmp/sihu.tif'
# shp = "../preprocess_20150122/final/Sihu"
# output = "../output/clip"

raster = '../../crop_img/normal/sihu_tmp.tif'

In [4]:
# Load the source data as a gdalnumeric array
srcArray = gdalnumeric.LoadFile(raster)

# Also load as a gdal image to get geotransform 
# (world file) info
srcImage = gdal.Open(raster)
geoTrans = srcImage.GetGeoTransform()

In [5]:
print srcArray.shape
print geoTrans

(3, 4280, 5526)
(164941.602510527, 2.0, 0.0, 2620419.960208477, 0.0, -2.0)


In [6]:
# image_data = srcArray
# image_data = srcArray.reshape(4280,5526,3)
# image_data = srcArray
image=Image.open('../../crop_img/normal/sihu_tmp.tif')
image.load()
image_data = np.asarray(image)

In [7]:
print image_data.max(axis=0).shape
print image_data.max(axis=1).shape
print image_data.max(axis=2).shape
image_data_bw = image_data.max(axis=2)
image_data_bw

(5526, 3)
(4280, 3)
(4280, 5526)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [8]:
non_empty_rows = np.where(image_data_bw.max(axis=0)>0)[0]
print non_empty_rows.shape
non_empty_rows

(521,)


array([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
         11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,
         22,   23,   24,   25,   26,   27,   28,   29,   30,   31,   32,
         33,   34,   35,   36,   37,   38,  272,  273,  274,  275,  276,
        277,  278,  279,  280,  281,  282,  283,  284,  285,  286,  287,
        288,  289,  290,  291,  292,  293,  294,  295,  296,  297,  298,
        299,  300,  301,  302,  367,  368,  369,  370,  371,  372,  373,
        374,  375,  376,  377,  378,  379,  380,  381,  382,  383,  384,
        385,  386,  387,  388,  389,  390,  391,  392,  393,  394,  395,
        396,  397,  398,  399,  400,  401,  402,  403,  404,  405,  406,
        407,  408,  409,  410,  411,  412,  413,  414,  415,  416,  417,
        418,  419,  420,  421,  422,  423,  424,  425,  426,  427,  428,
        429,  430,  431,  432,  433,  434,  435,  436,  437,  438,  439,
        440,  441,  847,  848,  849,  850,  851,  8

In [9]:
non_empty_columns = np.where(image_data_bw.max(axis=1)>0)[0]
print non_empty_columns.shape
non_empty_columns

(290,)


array([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
         11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   21,
         22,   23,   24,   25,   26,   27,   28,   29,   30,   31,   32,
         33,   34,   35,   36,   37,   38,   39,   40,   41,   42,   43,
         44,   45,   46,   47,   48, 1293, 1294, 1295, 1296, 1297, 1298,
       1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309,
       1310, 1311, 1312, 1313, 1314, 1315, 1316, 1317, 1318, 1319, 1320,
       1321, 1322, 1323, 1324, 1325, 1326, 1327, 1328, 1329, 1456, 1457,
       1458, 1459, 1460, 1461, 1462, 1463, 1464, 1465, 1466, 1467, 1468,
       1469, 1470, 1471, 1472, 1473, 1474, 1475, 1476, 1477, 1478, 1479,
       1480, 1481, 1482, 1483, 1484, 1485, 1486, 1487, 1488, 1489, 1490,
       1491, 1492, 1595, 1596, 1597, 1598, 1599, 1600, 1601, 1602, 1603,
       1782, 1783, 1784, 1785, 1786, 1787, 1788, 1789, 1790, 1791, 1792,
       1793, 1794, 1795, 1796, 1797, 1856, 1857, 18

In [1]:
# cropBox = (min(non_empty_columns), max(non_empty_columns), min(non_empty_rows), max(non_empty_rows))
# cropBox

In [10]:
# image_data_new = image_data[cropBox[0]:cropBox[1]+1, cropBox[2]:cropBox[3]+1 , :]
image_data_new = image_data[1293:1329, 0:38 , :]
new_image = Image.fromarray(image_data_new)
new_image.save('QQQ.tiff')